In [45]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np


# getting info from sports reference

In [36]:
def get_winners (link):
    print (link)
    page = requests.get(link)
    #print('here')
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    i=0
    k=0
    winners=[]
    for thing in list(range(1,100)):
        if k == 22:
            break
        elif k == 0 or k== 1 or k==2 or k==3 or k==5 or k==6 or k==7 or k==8 or k==10 or k==11 or k==12 or k==13 or k==15 or k==16 or k==17 or k==18 or k==20 or k==21:
            try:
                winners.append(idk[k].select('div')[i].select('div[class="winner"]')[0].select('span')[0].string)
                i+=3
            except:
                i=0
                k+=1
                continue
        else:
            i=0
            k+=1
            continue
    winners_int=[]
    for thing in winners:
        winners_int.append(int(thing))
    return winners_int

In [37]:
#function for getting the teams
#gets the teams that played each other all together
def get_teams (link,k):
    page = requests.get(link)
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    i=1
    maybe = []
    for thing in list(range(1,100)):
        if i%3 !=0:
            try:
                maybe.append(idk[k].select('div')[i].select('span')[0].string)
                i+=1
            except:
                i+=1
                continue
        else:
            i+=1
            continue
    return maybe



In [38]:
#uses the function to split up the teams that played each other into 2 separate lists
def split_up_teams (link):
    All_teams = []
    for thing in list(range(0,23)):
        new=get_teams (link,thing)
        All_teams.extend(new)
    All_teams.pop(-1)
    All_teams.pop(30)
    All_teams.pop(60)
    All_teams.pop(90)
    All_teams.pop(120)
    first_team = All_teams[0::2]
    second_team = All_teams[1::2]
    first_team_int = []
    for thing in first_team:
        first_team_int.append(int(thing))
    second_team_int = []
    for thing in second_team:
        second_team_int.append(int(thing))
    first_team_pd = pd.DataFrame(first_team_int)
    second_team_pd = pd.DataFrame(second_team_int)
    point_difference = first_team_pd-second_team_pd
    point_difference_list=point_difference.values.tolist()
    point_diff_abs = []
    for thing in point_difference_list:
        point_diff_abs.append(abs(thing[0]))
    return first_team_int,second_team_int,point_diff_abs



In [39]:
def get_team_names (link,k):
    page = requests.get(link)
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    maybe = []
    for thing in list(range(0,100)):
        try:
            maybe.append(idk[k].select('a[href^=/cbb/schools]')[thing].string)
        except:
            continue
    return maybe



In [40]:
def get_teams_get (link):
    team_names = []
    for thing in list(range(0,23)):
        new=get_team_names (link,thing)
        team_names.extend(new)
    team_names.pop(-1)
    team_names.pop(30)
    team_names.pop(60)
    team_names.pop(90)
    team_names.pop(120)
    return team_names



In [41]:
def split_up_teams_2 (link):
    All_teams = []
    for thing in list(range(0,23)):
        new=get_teams (link,thing)
        All_teams.extend(new)
    All_teams.pop(-1)
    All_teams.pop(30)
    All_teams.pop(60)
    All_teams.pop(90)
    All_teams.pop(120)
    return All_teams

In [42]:
#add in the year
#make the 9999

In [43]:
def everything (link):
    first_team_int = []
    second_team_int = []
    point_diff_abs = []
    winners_int = []

    winners=get_winners (link)
    three_lists=split_up_teams (link)
    first_team_int.extend(three_lists[0])
    second_team_int.extend(three_lists[1])
    point_diff_abs.extend(three_lists[2])
    winners_int.extend(winners)
    #time.sleep(3)
    #print(len(first_team_int), len(second_team_int),len(point_diff_abs), len(winners_int))
    df = pd.DataFrame({"first_team": first_team_int, "second_team": second_team_int,
                       "winner" : winners_int, "point_difference": point_diff_abs})
    result = []
    i = 0
    for thing in list(df['winner']):
        #print(f'this is {i}')
        if list(df['first_team'])[i] > list(df['second_team'])[i] and list(df['first_team'])[i] == list(df['winner'])[i]:
            #print(f'this is if1')
            result.append(2)
        elif list(df['second_team'])[i] > list(df['first_team'])[i] and list(df['second_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif1')
            result.append(2)
        elif list(df['first_team'])[i] < list(df['second_team'])[i] and list(df['first_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif2')
            result.append(1)
        elif list(df['second_team'])[i] < list(df['first_team'])[i] and list(df['second_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif3')
            result.append(1)
        else:
            result.append(0)
        i+=1
    df['result']=result
    newdf2 = pd.DataFrame(np.repeat(df.values,2,axis=0))
    newdf2.columns = df.columns
    team_names=get_teams_get (link)
    All_teams=split_up_teams_2(link)
    rankings= []
    for thing in All_teams:
        #print(thing)
        rankings.append(int(thing))
    newdf2['team_name'] = team_names
    newdf2['team_rank'] = rankings
    win = []
    i = 0
    for thing in list(newdf2['team_rank']):
        if list(newdf2['first_team'])[i] == list(newdf2['second_team'])[i]:
            win.append(9999)
        elif list(newdf2['team_rank'])[i] == list(newdf2['winner'])[i]:
            win.append(1)
        elif list(newdf2['team_rank'])[i] != list(newdf2['winner'])[i]:
            win.append(0)
        i+=1
    newdf2['win']=win
    #if the team has a lower seed number, it is given '1', higher seed number '0'
    higher_or_lower_seed = []
    i = 0
    for thing in list(newdf2['team_rank']):
        if list(newdf2['first_team'])[i] == list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(9999)
        elif list(newdf2['team_rank'])[i] == list(newdf2['first_team'])[i] and list(newdf2['first_team'])[i] < list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(1)
        elif list(newdf2['team_rank'])[i] == list(newdf2['first_team'])[i] and list(newdf2['first_team'])[i] > list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(0)
        elif list(newdf2['team_rank'])[i] == list(newdf2['second_team'])[i] and list(newdf2['second_team'])[i] < list(newdf2['first_team'])[i]:
            higher_or_lower_seed.append(1)
        elif list(newdf2['team_rank'])[i] == list(newdf2['second_team'])[i] and list(newdf2['second_team'])[i] > list(newdf2['first_team'])[i]:
            higher_or_lower_seed.append(0)
        i+=1
    newdf2['higher_or_lower_seed']=higher_or_lower_seed
    #newdf2.to_csv(f'{link}')
    newdf2.rename(columns = {'team_name':'name'}, inplace = True)
    newdf2['index']=list(range(0,126))
    return newdf2

In [46]:
newdf2=everything ('https://www.sports-reference.com/cbb/postseason/1996-ncaa.html')

https://www.sports-reference.com/cbb/postseason/1996-ncaa.html


In [47]:
year = []
for thing in newdf2['first_team']:
    year.append(1996)

In [48]:
newdf2['year']=year

In [206]:
newdf2.to_csv('sports_reference_2019')

# getting infor from ESPN

In [49]:
def get_score_info (links):
    names = []
    PTS = []
    FG_perc = []
    _3p_perc = []
    FT_perc = []
    for link in links:
        print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    PTS.append(float(thing.select('td')[3].string))
                    FG_perc.append(float(thing.select('td')[5].string))
                    _3p_perc.append(float(thing.select('td')[7].string))
                    FT_perc.append(float(thing.select('td')[9].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    PTS.append(float(idk2[i].select('td')[3].string))
                    FG_perc.append(float(idk2[i].select('td')[5].string))
                    _3p_perc.append(float(idk2[i].select('td')[7].string))
                    FT_perc.append(float(idk2[i].select('td')[9].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    PTS.append(float(thing.select('td')[3].string))
                    FG_perc.append(float(thing.select('td')[5].string))
                    _3p_perc.append(float(thing.select('td')[7].string))
                    FT_perc.append(float(thing.select('td')[9].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    PTS.append(float(idk[i].select('td')[3].string))
                    FG_perc.append(float(idk[i].select('td')[5].string))
                    _3p_perc.append(float(idk[i].select('td')[7].string))
                    FT_perc.append(float(idk[i].select('td')[9].string))
                    i+=1
                except:
                    continue
                    i+=1
            
            


            
    return names, PTS, FG_perc, _3p_perc, FT_perc

In [50]:
list_years = []
i = 1996
for num in range(0,1):
    list_years.append(str(i))
    i+=1

In [51]:
links = []
for thing in list_years:
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing)
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/41')
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/81')
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/121')
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/161')
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/201')
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/241')
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/281')
    links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/321')


In [52]:
names, PTS, FG_perc, _3p_perc, FT_perc = get_score_info (links)

http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/1996
http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/1996/count/41
http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/1996/count/81
http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/1996/count/121
http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/1996/count/161
http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/1996/count/201
http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/1996/count/241
http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/1996/count/281
http://www.espn.com/mens-college-basketball/statistics/team/

In [53]:
D = {'Name':names, 'PTS':PTS, 'FG_perc':FG_perc, '_3p_perc':_3p_perc, 'FT_perc':FT_perc}


In [54]:
df2 = pd.DataFrame(data=D)


In [55]:
year = []
for thing in df2['Name']:
    year.append(1996)
    

In [56]:
df2['year']=year

In [255]:
df2.to_csv('ESPN_2019')

# Merging espn and sports reference

In [57]:
possibly2 = pd.merge(newdf2,df2, left_on = 'name', right_on = 'Name')

In [58]:
possibly2 = possibly2.sort_values(by ='index' )

In [770]:
possibly2 = possibly2.sort_values(by ='index' )
possibly2

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index,year_x,Name,PTS,FG_perc,_3p_perc,FT_perc,year_y
0,1,16,1,15,1,Maryland,1,1,1,0,2002,Maryland,85.3,0.483,0.370,0.710,2002
6,1,16,1,15,1,Siena,16,0,0,1,2002,Siena,69.6,0.415,0.347,0.742,2002
7,8,9,8,1,1,Wisconsin,8,1,1,2,2002,Wisconsin,66.9,0.438,0.352,0.719,2002
9,5,12,12,7,2,Marquette,5,0,1,4,2002,Marquette,73.3,0.474,0.381,0.692,2002
10,5,12,12,7,2,Tulsa,12,1,0,5,2002,Tulsa,80.4,0.477,0.406,0.751,2002
12,4,13,4,9,1,Kentucky,4,1,1,6,2002,Kentucky,76.6,0.448,0.309,0.670,2002
15,4,13,4,9,1,Valparaiso,13,0,0,7,2002,Valparaiso,78.0,0.451,0.350,0.665,2002
16,6,11,11,5,2,Texas Tech,6,0,1,8,2002,Texas Tech,79.5,0.468,0.378,0.684,2002
17,6,11,11,5,2,Southern Illinois,11,1,0,9,2002,Southern Illinois,75.9,0.463,0.372,0.621,2002
20,3,14,3,11,1,Georgia,3,1,1,10,2002,Georgia,76.1,0.447,0.315,0.734,2002


In [59]:
def find_missing_values (blah):
    the_values = []
    for thing in list(range(0,126)):
        if thing in blah:
            continue
        else:
            print(thing)
            the_values.append(thing)
    return the_values

In [60]:
the_values=find_missing_values (list(possibly2['index']))

8
20
31
45


In [70]:
newdf2[(newdf2['index']==31)]


,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index,year
31,1,16,1,15,1,San Jose State,16,0,0,31,1996


In [918]:
pd.set_option('display.max_rows', 400)
df2.Name

0                       Alabama
1                       Gonzaga
2                  Wright State
3                          Duke
4            Eastern Washington
5                     Green Bay
6                           LSU
7                       Belmont
8                        Dayton
9            California Baptist
10                     Winthrop
11                          BYU
12             Central Michigan
13            Stephen F. Austin
14                 Oral Roberts
15                 South Dakota
16            Sam Houston State
17              Houston Baptist
18                      Buffalo
19                      McNeese
20             Western Carolina
21                       Auburn
22                         Iowa
23                Georgia State
24                   New Mexico
25                         UTSA
26           South Dakota State
27                      Arizona
28                    Creighton
29             St. Francis (PA)
30             Coastal Carolina
31      

In [74]:
df2

,Name,PTS,FG_perc,_3p_perc,FT_perc,year
0,Alabama,83.3,0.443,0.355,0.696,1996
1,Gonzaga,87.6,0.512,0.385,0.685,1996
2,Wright State,82.4,0.465,0.372,0.688,1996
3,Duke,82.6,0.477,0.355,0.695,1996
4,Eastern Washington,81.3,0.461,0.353,0.695,1996
5,Green Bay,81.5,0.461,0.373,0.751,1996
6,LSU,80.8,0.472,0.319,0.773,1996
7,Belmont,81.2,0.480,0.360,0.711,1996
8,Dayton,80.6,0.521,0.369,0.730,1996
9,California Baptist,80.7,0.450,0.382,0.778,1996


In [80]:
df2[(df2['Name']=='San José St')]

,Name,PTS,FG_perc,_3p_perc,FT_perc,year
196,San José St,70.6,0.393,0.3,0.719,1996


In [65]:
#still working on this
def add_to_df (name1, num,df):
    maybe=df2[(df2['Name']==f'{name1}')]
    maybe2=newdf2[(newdf2['index']==num)]
    dict = {'first_team':list(maybe2['first_team'])[0],'second_team':list(maybe2['second_team'])[0],
            'winner':list(maybe2['winner'])[0],
            'point_difference':list(maybe2['point_difference'])[0],'result':list(maybe2['result'])[0],
            'name':list(maybe2['name'])[0],
            'team_rank':list(maybe2['team_rank'])[0],'win':list(maybe2['win'])[0],
            'higher_or_lower_seed':list(maybe2['higher_or_lower_seed'])[0],
            'index':list(maybe2['index'])[0],'year_x':list(maybe['year'])[0], 'Name': list(maybe['Name'])[0],
            'PTS':list(maybe['PTS'])[0],'year_y':list(maybe['year'])[0],
            'FG_perc':list(maybe['FG_perc'])[0],'_3p_perc':list(maybe['_3p_perc'])[0],
            'FT_perc':list(maybe['FT_perc'])[0],'year_y':list(maybe['year'])[0]}
    possibly2=df.append(dict, ignore_index=True)
    return possibly2

In [116]:
pd.set_option('display.max_rows', 400)

possibly2 = possibly2.sort_values(by ='index' )
possibly2

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index,year_x,Name,PTS,FG_perc,_3p_perc,FT_perc,year_y
0,1,16,1,15,1,UMass,1,1,1,0,1996,UMass,68.7,0.426,0.324,0.691,1996
1,1,16,1,15,1,UCF,16,0,0,1,1996,UCF,69.0,0.426,0.315,0.708,1996
2,8,9,9,1,2,Bradley,8,0,1,2,1996,Bradley,71.7,0.443,0.374,0.730,1996
3,8,9,9,1,2,Stanford,9,1,0,3,1996,Stanford,70.1,0.471,0.372,0.692,1996
4,5,12,12,7,2,Penn State,5,0,1,4,1996,Penn State,76.7,0.444,0.332,0.694,1996
5,5,12,12,7,2,Arkansas,12,1,0,5,1996,Arkansas,74.0,0.437,0.320,0.727,1996
6,4,13,4,9,1,Marquette,4,1,1,6,1996,Marquette,77.2,0.426,0.378,0.738,1996
7,4,13,4,9,1,Monmouth,13,0,0,7,1996,Monmouth,70.5,0.405,0.333,0.726,1996
8,6,11,6,5,1,UNC,6,1,1,8,1996,North Carolina,70.6,0.413,0.291,0.663,1996
124,6,11,6,5,1,UNC,6,1,1,8,1996,North Carolina,70.6,0.413,0.291,0.663,1996


In [115]:
for thing in the_values:
    print(thing)
    the_name=list(newdf2[(newdf2['index']==thing)]['name'])[0]
    ESPN_name=ESPN_sports_reference_dict[f"{the_name}"]
    possibly2=add_to_df (ESPN_name, thing,possibly2)
    

8
20
31
45


# Making dictionary to reference sports reference and ESPN

In [81]:
ESPN_sports_reference_dict = {}

In [82]:
ESPN_sports_reference_dict["Alabama-Birmingham"] = "UAB"

In [83]:
ESPN_sports_reference_dict["ETSU"] = "East Tennessee State"

In [84]:
ESPN_sports_reference_dict["UNC"] = "North Carolina"

In [85]:
ESPN_sports_reference_dict["St. Joseph's"] = "Saint Joseph's"

In [86]:
ESPN_sports_reference_dict["Pitt"] = "Pittsburgh"

In [87]:
ESPN_sports_reference_dict["UC-Irvine"] = "UC Irvine"

In [88]:
ESPN_sports_reference_dict["Cal State Fullerton"] = "CSU Fullerton"

In [89]:
ESPN_sports_reference_dict["College of Charleston"] = "Charleston"

In [90]:
ESPN_sports_reference_dict["Miami (FL)"] = "Miami"

In [91]:
ESPN_sports_reference_dict["Loyola (IL)"] = "Loyola Chicago"

In [92]:
ESPN_sports_reference_dict["Penn"] = "Pennsylvania"

In [93]:
ESPN_sports_reference_dict["Mount St. Mary's"] = "Mt. St. Mary's"

In [94]:
ESPN_sports_reference_dict["ETSU"] = "East Tennessee State"

In [95]:
ESPN_sports_reference_dict["UC-Davis"] = "UC Davis"

In [96]:
ESPN_sports_reference_dict["Hawaii"] = "Hawai'i"

In [97]:
ESPN_sports_reference_dict["Cal State Bakersfield"] = "CSU Bakersfield"

In [98]:
ESPN_sports_reference_dict["Albany (NY)"] = "Albany"

In [99]:
ESPN_sports_reference_dict["St. John's (NY)"] = "St. John's"

In [100]:
ESPN_sports_reference_dict["Alabama-Birmingham"] = "UAB"

In [101]:
ESPN_sports_reference_dict["Detroit"] = "Detroit Mercy"

In [102]:
ESPN_sports_reference_dict["UCSB"] = "UC Santa Barbara"

In [103]:
ESPN_sports_reference_dict["St. Peter's"] = "Saint Peter's"

In [104]:
ESPN_sports_reference_dict["Cal State Northridge"] = "CSU Northridge"

In [105]:
ESPN_sports_reference_dict["Cal State Fullerton"] = "CSU Fullerton"

In [106]:
ESPN_sports_reference_dict["Texas-Arlington"] = "UT Arlington"

In [107]:
ESPN_sports_reference_dict["Texas A&M-Corpus Christi"] = "Texas A&M-CC"

In [108]:
ESPN_sports_reference_dict["Southeastern Louisiana"] = "SE Louisiana"

In [109]:
ESPN_sports_reference_dict["McNeese State"] = "McNeese"

In [110]:
ESPN_sports_reference_dict["Prairie View"] = "Prairie View A&M"

In [111]:
ESPN_sports_reference_dict["Nicholls State"] = "Nicholls"

In [112]:
ESPN_sports_reference_dict["San Jose State"] = "San José St"

In [113]:
ESPN_sports_reference_dict["Louisiana-Monroe"] = "UL Monroe"

In [114]:
ESPN_sports_reference_dict["Saint Francis (PA)"] = "St. Francis (PA)"

# Putting stuff into MySQl so I can use the LIKE operator and merge dataframes

In [193]:
import mysql.connector
import stuff
cnx = mysql.connector.connect(
    host = stuff.host,
    user = stuff.user,
    passwd = stuff.password)

In [194]:
cursor = cnx.cursor()

In [140]:
#insert_stm = """CREATE DATABASE march_madness;"""
#cursor.execute(insert_stm)
#cnx.commit()

In [195]:
cnx.database = 'march_madness'


In [142]:
df1.head()

,name,PTS,FG_perc,_3p_perc,FT_perc
0,VMI,87.9,0.449,0.359,0.709
1,Oakland,85.6,0.494,0.379,0.720
2,Washington,83.5,0.471,0.371,0.667
3,Long Island University,82.6,0.466,0.366,0.645
4,Kansas,82.4,0.514,0.387,0.681


In [172]:
create_table = """
CREATE TABLE ESPN
(name VARCHAR(225),
PTS FLOAT,
FG_perc FLOAT,
_3p_perc FLOAT,
FT_perc FLOAT);
"""

In [173]:
cursor.execute(create_table)


In [160]:
all_data = []
i=0
for thing in (list(df1['name'])):
    new_tuple = ((list(df1['name']))[i], (list(df1['PTS']))[i], (list(df1['FG_perc']))[i], (list(df1['_3p_perc']))[i],
                 (list(df1['FT_perc']))[i])
    all_data.append(new_tuple)
    i+=1

In [174]:

insert_stm = """INSERT INTO ESPN (name, PTS, FG_perc, _3p_perc, FT_perc) 
VALUES (%s, %s, %s, %s, %s)"""
cursor.executemany(insert_stm, all_data)
cnx.commit()

In [202]:
create_table = """
CREATE TABLE sports_reference
(first_team INT,
second_team INT,
winner INT,
point_difference INT,
result INT,
name VARCHAR(225),
team_rank INT,
win INT,
relative_seed INT,
index_made INT);
"""

In [203]:
cursor.execute(create_table)


In [204]:
newdf2.head(1)

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index
0,1,16,1,15,1,Ohio State,1,1,1,0


In [208]:
all_data = []
i=0
for thing in (list(newdf2['name'])):
    new_tuple = ((list(newdf2['first_team']))[i], (list(newdf2['second_team']))[i], (list(newdf2['winner']))[i],
                 (list(newdf2['point_difference']))[i], (list(newdf2['result']))[i], (list(newdf2['name']))[i], (list(newdf2['team_rank']))[i],
                 (list(newdf2['win']))[i], (list(newdf2['higher_or_lower_seed']))[i], (list(newdf2['index']))[i]
                )
    all_data.append(new_tuple)
    i+=1

In [209]:
all_data[0]

(1, 16, 1, 15, 1, 'Ohio State', 1, 1, 1, 0)

In [210]:
insert_stm = """INSERT INTO sports_reference (first_team,
second_team,
winner,
point_difference,
result,
name,
team_rank,
win,
relative_seed,
index_made) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
cursor.executemany(insert_stm, all_data)
cnx.commit()

In [211]:
list(df1['name'])

,name,PTS,FG_perc,_3p_perc,FT_perc
0,VMI,87.9,0.449,0.359,0.709


In [212]:
df1_sorted = df1.sort_values(by ='name' )

In [216]:
df1_sorted[df1_sorted(['name']='ohio')]

SyntaxError: keyword can't be an expression (<ipython-input-216-a288dbd8a224>, line 1)

# testing stuff

In [272]:
data = {'Name':['Tom', 'nick', 'krish', 'jack'], 'Age':[20, 21, 19, 18], 'index_made':[0,1,2,3]}

In [274]:
data1 = pd.DataFrame(data)

In [275]:
data1

,Name,Age,index_made
0,Tom,20,0
1,nick,21,1
2,krish,19,2
3,jack,18,3


In [285]:
dat = {'Name':['Tomas', 'nickolas', 'krishas', 'jack'], 'p3':[30,40,50,60], 'ft':[8,7,3,2]}

In [286]:
data2 = pd.DataFrame(dat)

In [287]:
data2

,Name,p3,ft
0,Tomas,30,8
1,nickolas,40,7
2,krishas,50,3
3,jack,60,2


In [288]:
ugh=pd.merge(data1,data2,how='inner',on='Name')

In [289]:
ugh

,Name,Age,index_made,p3,ft
0,jack,18,3,60,2


In [292]:
maybe=data2[(data2['Name']=='Tomas')]
maybe

,Name,p3,ft
0,Tomas,30,8


In [293]:
maybe2=data1[(data1['Name']=='Tom')]
maybe2

,Name,Age,index_made
0,Tom,20,0


In [300]:
dict = {'Name':list(maybe2['Name'])[0],'Age':list(maybe2['Age'])[0],'index_made':list(maybe2['index_made'])[0],
        'p3':list(maybe['p3'])[0],'ft':list(maybe['ft'])[0]}

In [305]:
ugh=ugh.append(dict, ignore_index=True)

In [306]:
ugh

,Name,Age,index_made,p3,ft
0,jack,18,3,60,2
1,Tom,20,0,30,8


In [299]:
list(maybe2['Name'])[0]

'Tom'

In [ ]:
def add_to_df (name1, name2):
    maybe=data2[(data2['Name']==f'{name1}')]
    maybe2=data1[(data1['Name']==f'{name2}')]
    dict = {'Name':list(maybe2['Name'])[0],'Age':list(maybe2['Age'])[0],'index_made':list(maybe2['index_made'])[0],
        'p3':list(maybe['p3'])[0],'ft':list(maybe['ft'])[0]}
    ugh=ugh.append(dict, ignore_index=True)
